In [ ]:
import argparse
import requests

import psycopg2
from psycopg2 import extras
from psycopg2.errors import DuplicateTable, OperationalError

In [ ]:
def parse_arguments():
    """Parse command line arguments.

    Args:
        None
    Returns:
        args (argparse.Namespace) : Arguments object
    """
    parser = argparse.ArgumentParser()

    parser.add_argument('-k', '--api_key', type=str, help="API key to use", required=True)
    parser.add_argument('-sd', '--start', type=str, help="Start date y-m-d", required=True)
    parser.add_argument('-ed', '--end', type=str, help="End date y-m-d", required=True)
    parser.add_argument('-to', '--save_to', type=str, help="Folder to save the data", default=".")
    parser.add_argument('-l', '--chunk_len', type=int, help="Number of rows in downloaded chunk", default=5000)

    # Parse the arguments
    args = parser.parse_args()
    return args 

In [ ]:
def generate_url(
    start_date, 
    end_date, 
    offset, 
    chunk_len, 
    api_key,
    use_api
):
    url_demand = f'https://api.eia.gov/v2/electricity/rto/' +\
    f'daily-region-sub-ba-data/data/?frequency=daily&data[0]=value' + \
    f'&facets[subba][]=ZONJ&start={start_date}&end={end_date}' + \
    f'&sort[0][column]=period&sort[0][direction]=desc&offset={offset}' + \
    f'&length={chunk_len}&api_key={api_key}'

    url_weather_historical = f'https://archive-api.open-meteo.com/v1' + \
    f'/archive?latitude=52.52&longitude=13.41&start_date={start_date}' + \
    f'&end_date={end_date}&daily=weathercode,temperature_2m_max,' + \
    f'temperature_2m_min,temperature_2m_mean' + \
    f'&timezone=America%2FNew_York'
    
    url_weather_latest = f'https://api.open-meteo.com/v1/forecast?' + \
    f'latitude=52.52&longitude=13.41&hourly=temperature_2m&daily=' + \
    f'temperature_2m_max,temperature_2m_min&timezone=America%2F' + \
    f'New_York&past_days=7'

    api_urls = {
        'eia': url_demand,
        'weather_hist': url_weather_historical,
        'weather_latest': url_weather_latest
    }

    return api_urls.get(use_api, None)

In [ ]:
def request_data(url):
    response = requests.get(url)

    if response.status_code == 200:  # Success
        return response.json()
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return None

In [ ]:
def download_data(url):
    
    print("Getting data...")
    
    data = request_data(url)
    if data:
        try:
            if 'response' in data.keys():
                print(data['response']['data'][0])
        except:
            pass

In [ ]:
def create_table(
    tabname: str,
    dbname: str,
    user: str,
    password: str,
    host: int,
    port: int,
    connect_timeout: int
) -> None:
    valid_tab_names = ['demand', 'weather']
    assert tabname in valid_tab_names, "Enter a valid table name."
    
    conn_params = {
        'dbname': dbname,
        'user': user,
        'password': password,
        'host': host,
        'port': port,
        'connect_timeout': connect_timeout
    }
    
    # Connect to your postgres DB
    try:
        conn = psycopg2.connect(**conn_params)
    
    except OperationalError as e:
        raise OperationalError(f"Error connecting to the database: {e}")

    cur = conn.cursor()

    demand_cols = '''
        period VARCHAR(100),
        subba VARCHAR(100),
        subba_name VARCHAR(100),	
        parent VARCHAR(100),	
        parent_name VARCHAR(100),	
        timezone VARCHAR(100),	
        value INTEGER,	
        value_units VARCHAR(100)
    '''

    weather_cols = '''
        time VARCHAR(100),
        temperature_2m_max	REAL,
        temperature_2m_min	REAL,
        temperature_2m_mean REAL
    '''

    tab_schema = {
        'demand': demand_cols,
        'weather': weather_cols
    }

    # Define the CREATE TABLE statement
    create_table_query = f'''
    CREATE TABLE {tabname} (
        id SERIAL PRIMARY KEY, 
        {tab_schema.get(tabname)}
    );
    '''
    
    try:
        # Execute the query
        cur.execute(create_table_query)

        # Commit the changes
        conn.commit()
    
    except DuplicateTable as e:
        cur.close()
        conn.close()
        raise DuplicateTable(f"The table {tabname} already exists.")
    
    # Close the cursor and connection
    cur.close()
    conn.close()

In [ ]:
def populate_table(
    data: list,
    tabname: str,
    dbname: str,
    user: str,
    password: str,
    host: int,
    port: int,
    connect_timeout: int
) -> None:
    
    conn_params = {
        'dbname': dbname,
        'user': user,
        'password': password,
        'host': host,
        'port': port,
        'connect_timeout': connect_timeout
    }
    
    # Connect to your postgres DB
    try:
        conn = psycopg2.connect(**conn_params)
    
    except OperationalError as e:
        raise OperationalError(f"Error connecting to the database: {e}")

    cur = conn.cursor()

    # Convert list of dictionaries to a list of tuples
    data_tuples = [tuple(d.values()) for d in data]
    col_names = ", ".join(data[0].keys()).replace("-", "_")

    # Insert data
    extras.execute_values(
        cur,
        f"INSERT INTO {tabname} ({col_names}) VALUES %s",
        data_tuples
    )

    conn.commit()
    cur.close()
    conn.close()

In [ ]:
params = {
        'tabname': 'demand',
        'dbname': 'db_demand',
        'user': 'dbuser',
        'password': '123',
        'host': 'localhost',
        'port': '5432',
        'connect_timeout': 5
    }
    
create_table(**params)

In [ ]:
# args = parse_arguments()
# api_key = args.api_key
# start_date = args.start
# end_date = args.end
# output_path = args.save_to
# chunk_len = args.chunk_len
# offset = 0
api_key = ''
start_date = '2018-07-01'
end_date = '2023-07-01'
chunk_len = 10000
offset = 0

urls = [
    f'https://api.eia.gov/v2/electricity/rto/' +\
    f'daily-region-sub-ba-data/data/?frequency=daily&data[0]=value' + \
    f'&facets[subba][]=ZONJ&start={start_date}&end={end_date}' + \
    f'&sort[0][column]=period&sort[0][direction]=asc&offset={offset}' + \
    f'&length={chunk_len}&api_key={api_key}',

    f'https://archive-api.open-meteo.com/v1' + \
    f'/archive?latitude=52.52&longitude=13.41&start_date={start_date}' + \
    f'&end_date={end_date}&daily=weathercode,temperature_2m_max,' + \
    f'temperature_2m_min,temperature_2m_mean' + \
    f'&timezone=America%2FNew_York',
    
    f'https://api.open-meteo.com/v1/forecast?' + \
    f'latitude=52.52&longitude=13.41&hourly=temperature_2m&daily=' + \
    f'temperature_2m_max,temperature_2m_min&timezone=America%2F' + \
    f'New_York&past_days=7',
]

In [ ]:
url = urls[0]
data = request_data(url)
data_list = data['response']['data']

params = {
        'data': data_list,
        'tabname': 'demand',
        'dbname': 'db_demand',
        'user': 'dbuser',
        'password': '123',
        'host': 'localhost',
        'port': '5432',
        'connect_timeout': 5
    }
populate_table(**params)